In [7]:
# Se descarga las librerias de nltk
# !pip install nltk
# !pip install stanza

# Se importa la libreria de pandas y nltk
import pandas as pd
import nltk
# import spacy
import stanza

# Se importa el conector para MongoBD
import pymongo

# Se descarga la WordNet desde nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('omw-1.4')
nltk.download('stopwords')
stanza.download('es')

# from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SpanishStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2022-05-11 16:47:44 INFO: Downloading default packages for language: es (Spanish)...
2022-05-11 16:47:47 INFO: File exists: C:\Users\adria\stanza_resources\es\default.zip
2022-05-11 16:47:55 INFO: Finished downloading models and saved to C:\Users\adria\stanza_resources.


In [8]:
data = pd.read_csv('data_total_con_codigo_001.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data[:2000]
data

,feedback,codigo
0,"De acuerdo a las gráficas elegidas, las mismas...",1866
1,Nombres de los paises,1867
2,Ninguna,1868
3,Creo que hace falta dar un color a cada carrer...,1874
4,"Unirlos en su solo gráfico, aunque cada uno es...",1875
...,...,...
1995,Mi trabajo también esta bien hecho.,12653
1996,Está igual al mio pero por motivos de que no p...,12654
1997,La imagen,12702
1998,realizó un adecuado uso de los materiales didá...,12705


In [9]:
stop_words = set(stopwords.words('spanish'))

In [18]:
nlp = stanza.Pipeline(lang='es',processors='tokenize,lemma', tokenize_pretokenized=True)
# nlp = spacy.load('es_core_news_sm')

list_lema = []
for i in range(len(data)):
    sentence = nltk.word_tokenize(re.sub("\!|\\\\|\'|\"", " ", data.iloc[i]['feedback']),"spanish")
    sentence = list(filter(lambda token: token not in string.punctuation, sentence))
    
    words = []
    for word in sentence:
        if word not in stop_words:
            words.append(nlp(word).sentences[0].words[0].lemma)
    
    list_lema.append({
        'codigo': int(data.iloc[i]['codigo']),
        'feedback_o': data.iloc[i]['feedback'],
        'feedback': ' '.join(words)
    })
    
list_lema[:20]

2022-05-11 17:07:17 WARNING: Language es package default expects mwt, which has been added
2022-05-11 17:07:17 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| lemma     | ancora  |

2022-05-11 17:07:17 INFO: Use device: cpu
2022-05-11 17:07:17 INFO: Loading: tokenize
2022-05-11 17:07:17 INFO: Loading: mwt
2022-05-11 17:07:17 INFO: Loading: lemma
2022-05-11 17:07:17 INFO: Done loading processors!


[{'codigo': 1866,
  'feedback_o': 'De acuerdo a las gráficas elegidas, las mismas estarian completas',
  'feedback': 'de acuerdo gráfica elegido mismo estarian completo'},
 {'codigo': 1867,
  'feedback_o': 'Nombres de los paises',
  'feedback': 'nombre pais'},
 {'codigo': 1868, 'feedback_o': 'Ninguna', 'feedback': 'ninguno'},
 {'codigo': 1874,
  'feedback_o': 'Creo que hace falta dar un color a cada carrera para poder diferenciarlas en la gráfica y además como ya mencione el nombre de una carrera esta incompleto aquel que dice \\"Ingeniería de prevención de Rie\\" y en la misma además falta el numero de estudiantes.',
  'feedback': 'creer hacer falta dar color cada carrera poder diferenciarlas gráfico además mencione nombre carrera incompleto aquel decir Ingeniería prevención rie mismo además falta numero estudiante'},
 {'codigo': 1875,
  'feedback_o': 'Unirlos en su solo gráfico, aunque cada uno esta muy bien desarrollado ',
  'feedback': 'unirlo solo gráfico aunque cada bien desarrol

In [6]:
# wnl = WordNetLemmatizer()
# data_final = pd.concat([data['codigo'], [wnl.lemmatize(i) for i in data['feedback']]], axis=1)
# data_final

MONGODB_HOST = '127.0.0.1'
MONGODB_PORT = '27017'

URI_CONNECTION = "mongodb://%s:%s/" % (MONGODB_HOST, MONGODB_PORT)

try:
    client = pymongo.MongoClient(URI_CONNECTION)
    client.server_info()
    print('OK -- Connected to MongoDB at server %s' % (MONGODB_HOST))
 
    database_entry = list_lema
    destination = 'LEMAS'
    bd = client['TEST']
    coleccion = bd[destination]
    print(coleccion)
    print
    coleccion.insert_many(database_entry)
    print("Data saved at %s collection in %s database: %s" % (destination, 'TEST', database_entry))
    client.close()
except Exception as error:
    print('Error with MongoDB connection: %s' % error)


OK -- Connected to MongoDB at server 127.0.0.1
Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'TEST'), 'LEMAS')
Error with MongoDB connection: cannot encode object: 1866, of type: <class 'numpy.int64'>
